In [545]:
import pandas as pd
import numpy as np

先確保每個欄位都沒有奇怪data

1. 先手動刪除除了這七個之外的欄位的資料
2. 檢查每個欄位的值是否都在domain裡面
    * space: 1, 2, 3, 4, 5, 6, 7, 8, 9, FZ, L, M, Y
    * action: JS, SS, FS, R, G, A, B, D, OP(對方問題)

In [546]:
df_IRI = pd.read_excel('Data\IRI VS POL.xlsx', sheet_name = 'IRI Set1')
df_IRI = df_IRI.dropna(how='all')
df_IRI.insert(1, 'Team', 'IRI')
df_IRI.insert(2, 'Rally', np.nan)
df_IRI.tail()

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
173,NaN,IRI,NaN,NaN,NaN,OP,NaN,NaN,1.0
174,NaN,IRI,NaN,22.0,Y,JS,NaN,NaN,NaN
175,0.0,IRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,IRI,NaN,22.0,5,D,1.0,NaN,NaN
177,0.0,IRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [547]:
df_POL = pd.read_excel('Data\IRI VS POL.xlsx', sheet_name = 'POL Set1')
df_POL = df_POL.dropna(how='all')
df_POL.insert(1, 'Team', 'POL')
df_POL.insert(2, 'Rally', np.nan)
df_POL.tail()

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
176,1.0,POL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,NaN,POL,NaN,17,8,R,NaN,1.0,NaN
178,NaN,POL,NaN,19,2,G,NaN,1.0,NaN
179,NaN,POL,NaN,16,2,A,NaN,NaN,1.0
180,0.0,POL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


合併兩個dataset
* df_1: 發球方
* df_2: 另一方
* i, j 分別代表兩邊目前的指標(?
---
要加入的欄位：rally_num, team


In [548]:
def get_rally_sequence(df_1, df_2, i, j, rally):
    df_combine = pd.DataFrame(columns=df_1.columns)
    # 發球方發球
    df_combine = df_combine.append(df_1.iloc[i])
    i += 1
    alternate = 1
    while(i < len(df_1)-1 and j < len(df_2)-1):
        if(alternate == 0): # df_1
            if(not pd.isna(df_1.iloc[i]['Is_blue_space'])):
                i += 1
            df_combine = df_combine.append(df_1.iloc[i])
            i += 1
            while(pd.isna(df_1.iloc[i]['Is_blue_space']) and 'S' not in str(df_1.iloc[i]['Action'])): # 遇到空格或再次發球前
                
                df_combine = df_combine.append(df_1.iloc[i])
                i += 1
                if(i >= len(df_1)-1):
                    break

            # 因為空格而停止
            if(not pd.isna(df_1.iloc[i]['Is_blue_space'])):
                alternate = 1
                continue

            # 因為再次發球而停止
            if('S' in df_1.iloc[i]['Action']):
                # 先確認另一隊在藍色空格之前的state是否全部記錄到
                while(df_2.iloc[j]['Is_blue_space'] != 1 and j < len(df_2)):
                    if(df_2.iloc[j]['Is_blue_space'] != 0):
                        df_combine = df_combine.append(df_2.iloc[j])
                    j += 1
                break

        elif(alternate == 1):   # df_2
            if(not pd.isna(df_2.iloc[j]['Is_blue_space'])):
                j += 1
            df_combine = df_combine.append(df_2.iloc[j])
            j += 1
            while(pd.isna(df_2.iloc[j]['Is_blue_space']) and 'S' not in df_2.iloc[j]['Action']): # 遇到空格或再次發球前
                
                df_combine = df_combine.append(df_2.iloc[j])
                j += 1
                if(j >= len(df_2)-1):
                    break

            if(not pd.isna(df_2.iloc[j]['Is_blue_space'])): # is space
                alternate = 0
                continue
            
            if('S' in df_2.iloc[j]['Action']):  # 再次發球 -> 下一round
                # 先確認另一隊在藍色空格之前的state是否全部記錄到
                while(df_1.iloc[i]['Is_blue_space'] != 1 and i < len(df_1)):
                    if(df_1.iloc[i]['Is_blue_space'] != 0):
                        df_combine = df_combine.append(df_1.iloc[i])
                    i += 1
                break
    if(i == len(df_1)-1):
        while(j < len(df_2)-1):
            df_combine = df_combine.append(df_2.iloc[j])
            j += 1
    if(j == len(df_2)-1):
        while(i < len(df_1)-1):
            df_combine = df_combine.append(df_1.iloc[i])
            i += 1

    df_combine['Rally'] = rally
    return df_combine, i, j

In [549]:
df_total_game = pd.DataFrame(columns=df_IRI.columns)

In [550]:
i = 0
j = 0
rally_num = 0
while(i < len(df_IRI)-1 or j < len(df_POL)-1):
    rally_num += 1
    print(i, j)
    if(df_IRI.iloc[i]['Is_blue_space'] == 1):   # 對方發球
        df_combine, j, i = get_rally_sequence(df_POL, df_IRI, j, i, rally_num)
    else:
        df_combine, i, j = get_rally_sequence(df_IRI, df_POL, i, j, rally_num)
    df_total_game = pd.concat([df_total_game, df_combine])
    
    print('\nrally: ' + str(rally_num))

0 0

rally: 1
3 4

rally: 2
6 8

rally: 3
12 13

rally: 4
21 22

rally: 5
24 28

rally: 6
25 30

rally: 7
29 33

rally: 8
31 34

rally: 9
35 42

rally: 10
39 45

rally: 11
40 47

rally: 12
44 50

rally: 13
49 58

rally: 14
53 60

rally: 15
57 63

rally: 16
61 66

rally: 17
68 75

rally: 18
71 77

rally: 19
78 83

rally: 20
81 87

rally: 21
86 92

rally: 22
99 100

rally: 23
104 106

rally: 24
105 108

rally: 25
107 109

rally: 26
110 113

rally: 27
114 116

rally: 28
122 125

rally: 29
124 126

rally: 30
127 130

rally: 31
134 135

rally: 32
138 138

rally: 33
141 142

rally: 34
149 152

rally: 35
151 153

rally: 36
152 155

rally: 37
154 157

rally: 38
160 160

rally: 39
162 162

rally: 40
165 168

rally: 41
169 171

rally: 42
172 175

rally: 43
174 176

rally: 44


In [551]:
df_total_game

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
0,NaN,IRI,1,2,Y,JS,NaN,NaN,NaN
1,NaN,POL,1,14,8,R,NaN,NaN,NaN
2,NaN,POL,1,19,2,G,NaN,NaN,NaN
3,NaN,POL,1,6,3,A,1.0,NaN,NaN
2,NaN,IRI,1,22,1,B,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
177,NaN,POL,44,17,8,R,NaN,1.0,NaN
178,NaN,POL,44,19,2,G,NaN,1.0,NaN
179,NaN,POL,44,16,2,A,NaN,NaN,1.0
175,0.0,IRI,44,NaN,NaN,NaN,NaN,NaN,NaN


In [552]:
drop_list = []
for idx, row in df_total_game.iterrows():
    if(not pd.isna(row['Is_blue_space'])):
        drop_list.append(idx)

In [553]:
df_total_game = df_total_game.drop(index=drop_list)
df_total_game = df_total_game.reset_index(drop=True)

In [554]:
df_total_game = df_total_game.drop(columns='Is_blue_space')

In [555]:
df_total_game.to_csv('set1_test.csv', index=False)

---

* Action + Attempts 成為一個新的Action欄位 (足以區分是否好球)
* Team、space、action、No.轉成 one-hot encoding形式

In [556]:
for idx, row in df_total_game.iterrows():
    if(pd.isna(row['Attempts'])):
        df_total_game.loc[idx, 'Action'] = row['Action'] + '_0'
    else:
        df_total_game.loc[idx, 'Action'] = row['Action'] + '_1'
df_total_game = df_total_game.drop(columns='Attempts')

In [557]:
# pd.get_dummies(df_total_game, columns=['Team', 'No.', 'Space', 'Action'])

In [558]:
for d in df_total_game.groupby('Rally'):
    print(d)

(1,   Team Rally No. Space Action  Errors  Score
0  IRI     1   2     Y   JS_0     NaN    NaN
1  POL     1  14     8    R_0     NaN    NaN
2  POL     1  19     2    G_0     NaN    NaN
3  POL     1   6     3    A_0     1.0    NaN
4  IRI     1  22     1    B_0     NaN    1.0)
(2,   Team Rally No. Space Action  Errors  Score
5  IRI     2   2     Y   JS_0     NaN    NaN
6  POL     2  14     7    R_0     NaN    NaN
7  POL     2  19     1    G_0     NaN    NaN
8  POL     2   6     3    A_0     NaN    1.0
9  IRI     2   2     8    D_0     1.0    NaN)
(3,    Team Rally No. Space Action  Errors  Score
10  POL     3  15     Y   JS_0     NaN    NaN
11  IRI     3   8     8    R_0     NaN    NaN
12  IRI     3  18     2    G_0     NaN    NaN
13  IRI     3   5     2    A_0     NaN    NaN
14  POL     3  19     5    D_0     NaN    NaN
15  POL     3  14     4    G_0     NaN    NaN
16  POL     3  16     1    A_0     NaN    1.0
17  IRI     3  17     3    B_0     1.0    NaN)
(4,    Team Rally No. Space Act